<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/introductionVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Méthode de vision par ordinateur "traditionnelles"

## Extraction de contours

In [14]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy
import PIL
from PIL import Image

def show(x):
    if len(x.shape)==2:
        plt.imshow(x, interpolation='nearest')
        return
    if len(x.shape)==3:
        visu = torch.stack([x[0],x[1],x[2]],dim=-1)
        plt.imshow(visu.numpy(), interpolation='nearest')
    else:
        show(torchvision.utils.make_grid(x))

def readimage(path):
    tmp = PIL.Image.open(path)
    x = torch.Tensor(numpy.asarray(tmp)).clone()
    if len(x.shape)==2:
        return x
    else:
        return torch.stack([x[:,:,0],x[:,:,1],x[:,:,2]],dim=0)


In [15]:
!wget https://httpmail.onera.fr/5/ab54f8bf5ce3bc61fe8dda498f109b76YvVQrY/astronaut.jpg
image = readimage("astronaut.jpg")
show(image)

--2024-03-12 16:12:05--  https://httpmail.onera.fr/5/ab54f8bf5ce3bc61fe8dda498f109b76YvVQrY/astronaut.jpg
Resolving httpmail.onera.fr (httpmail.onera.fr)... 144.204.16.9
Connecting to httpmail.onera.fr (httpmail.onera.fr)|144.204.16.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40344 (39K) [application/force-download]
Saving to: ‘astronaut.jpg.1’

astronaut.jpg.1     100%[===================>]  39.40K  --.-KB/s    in 0.1s    

2024-03-12 16:12:06 (282 KB/s) - ‘astronaut.jpg.1’ saved [40344/40344]



UnidentifiedImageError: cannot identify image file 'astronaut.jpg'

In [ ]:
show(image[0])

In [ ]:
import torch

def sobel_filter(image):
    """
    Applies a Sobel filter to a grayscale PyTorch tensor (image).

    Args:
        image (torch.Tensor): A grayscale image tensor of shape (batch_size, channels, height, width).

    Returns:
        torch.Tensor: A tensor of shape (batch_size, 2, height, width), where the first channel
                      represents the horizontal gradient and the second channel represents the vertical gradient.
    """

    if image.dim() != 4:
        raise ValueError("Input image must be a 4D tensor (batch_size, channels, height, width).")

    if image.shape[1] != 1:
        raise ValueError("Input image must be grayscale (single channel).")

    # Sobel filter kernels for horizontal and vertical gradients
    sobel_x = torch.tensor([[1, 0, -1],
                           [2, 0, -2],
                           [1, 0, -1]], dtype=image.dtype)[:, None, ...]
    sobel_y = torch.tensor([[1, 2, 1],
                           [0, 0, 0],
                           [-1, -2, -1]], dtype=image.dtype)[:, None, ...]

    # Ensure kernels are on the same device as the image
    sobel_x = sobel_x.to(image.device)
    sobel_y = sobel_y.to(image.device)

    # Apply convolution with each kernel
    horizontal_gradients = torch.nn.functional.conv2d(image, sobel_x, padding=1)
    vertical_gradients = torch.nn.functional.conv2d(image, sobel_y, padding=1)

    # Combine gradients (optional: magnitude or direction calculation)
    # gradients = torch.sqrt(horizontal_gradients**2 + vertical_gradients**2)  # Magnitude
    # OR
    # gradients = torch.atan2(vertical_gradients, horizontal_gradients)  # Direction

    return torch.cat((horizontal_gradients, vertical_gradients), dim=1)

# Example usage
image = torch.randn(1, 1, 28, 28)  # Example grayscale image of size (28, 28)
gradients = sobel_filter(image)
print(gradients.shape)  # Output: torch.Size([1, 2, 28, 28])
